<a href="https://colab.research.google.com/github/bgalerne/M2MAS_neural_networks/blob/main/M2MAS_Unet_segmentaiton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# U-net for crack segmentation
### Assignment for M2MAS "Réseaux de Neurones profonds pour l'Apprentissage"

*References:*
 * U-Net: Convolutional Networks for Biomedical Image Segmentation, Olaf Ronneberger, Philipp Fischer, Thomas Brox, Medical Image Computing and Computer-Assisted Intervention (MICCAI), Springer, LNCS, Vol.9351: 234--241, 2015, available at [arXiv:1505.04597](http://arxiv.org/abs/1505.04597)
 * An 'All Terrain' Crack Detector Obtained by Deep Learning on Available Databases, Sébastien Drouyer, Image Processing On Line, 10 (2020), pp. 105–123. https://doi.org/10.5201/ipol.2020.282


### Modalities:
This assignment has to be send by email to bruno.galerne@univ-orleans.fr before Sunday March 7, 23:59.
Each question should be answered by providing a new section of the notebook (starting with a text cell ```# Question 1```) with all necessary code.
Code must be commented and introduced with a text cell. Results must be discussed in a text cell. In case of difficulty, you can also discuss errors in a text cell.

Each training of the neural network should be limited to 20 epochs. Note that the computation time will be arround 2 hours on a K80 GPU, and faster with newer GPUs.

### Questions:
First read and run the proposed notebook to get a first version of the network.
1. **Performance evaluation:** Explain (in a text cell) what is computed by
```
valscores = test_performance(unet, valset)
testscores = test_performance(unet, testset)
```

1. **First training:** 
  * Improve the proposed code by adding the capacity to save the trained model to disk and load the model from disk (see https://pytorch.org/tutorials/beginner/saving_loading_models.html#saving-loading-model-for-inference)
  * Train the model for 20 epochs (only 2 while developing the code...). 
  * Store the training loss at the end of each epoch. At the end of training plot the epoch loss for each epoch.
  * Download the model parameters and/or save them to your google drive for a reference.
  * Reload this models as ```unet_ref``` and check the reload is OK.
  * Evaluate and report its performance scores using ```test_performance``` on the testset.
1. **Training with data augmentation:**
  * Explain in a few sentences the concept and interest of data augmentation. 
  * Discuss a data augmentation strategy for the proposed dataset and implement it within ```class CRACK500(Dataset):```
  * Retrain the U-net with data augmentation (starting from a random initialization) for 20 epochs.
  * Save and reload the model as  ```unet_data_aug```.
  * Compare with ```unet_ref``` regarding training loss and performance on test set.
1. **Early stopping:** Using the data augmentation of the previous question:
  * At each epoch, evaluate the performance on the validation set and save the network with the best F1 score on validation set.
  * Save and reload the retained model as  ```unet_data_aug_early_stop```.
  * Compare to other models regarding performance on the test set. What is the best model among the three trainings?

**Bonus question (on 3 points/20): Training using BatchNormalization layers:**
  * Recall the principle of BatchNormalization layers
  * Define a new class Unet_bn network model by adding one batch normalization layer before each max pool and up-convolution.
  * Train this new network using data augmentation. Save it.
  * Compare the performance of this new network. Make sure to be in evaluation mode for testing.



# Provided notebook (not to be changed)
*This whole section should not be changed.
You may copy and change cells into your sections.
You can suppose that the cells of the provided notebook have been executed before your answers (no need to copy everything, only the changes)* 

## Check that the notebook has an active GPU:
(otherwise go to Edit->Notebook properties ->...)

In [ ]:
!nvidia-smi

In [ ]:
!ls

Download CRACK500 datatset:
 * Links are here:
https://drive.google.com/drive/folders/1y9SxmmFVh0xdQR-wdchUmnScuWMJ5_O-
 * Reference work is here:
https://github.com/fyangneil/pavement-crack-detection

CRACK500
 * Format :640 x 360(60kb, jpeg)
 * Dataset Size : Total 3368 Images - Train(1896 Images), Val(348 Images), Test(1124 Images)

In [ ]:
# Original data, not used:
# # traindata.zip
# !gdown --id 1iOaEWPJDY4U5s0BOOb1fIjcHWDc7FhX-
# # testdata.zip
# !gdown --id 1C7pj7BP32Qqpm6q8QmirJfjc37rhPdZh
# # valdata.zip 
# !gdown --id 1fUd1BBh2BRWdbTe5p8R5NK-eBzhZYumw

# Images:
# traincrop.zip
!gdown --id 1Qgex6YDpQ0yRrD8JFtNggYN30Ner3rY8
# testcrop.zip
!gdown --id 1u7wuaQHWWUtF5ON0MhGXcjwbfItniIK5
# valcrop.zip 
!gdown --id 16eqMf5E9C-eQxGl-ATCQjm4zfLkCkxNn
# Download .txt files describing datasets:
# train.txt
!gdown --id 1278gNvcfmGQ3yPw3i9v6ATkLxm0-5Iyk
# test.txt
!gdown --id 1Sq_jnVulc3cspnSXzEvBxpy6AV8D5FwD
# val.txt 
!gdown --id 1zrLC6qjJNYOOeRGrlBnZHgv9BKZMj2f2

Unzip data (-q=quietly):

In [ ]:
# !unzip traindata.zip
# !unzip testdata.zip
# !unzip valdata.zip
!unzip -q traincrop.zip
!unzip -q testcrop.zip
!unzip -q valcrop.zip
!ls

#PyTorch Dataloaders:

Create a data loader for the train, test, val datasets.
We follow mostly:
WRITING CUSTOM DATASETS, DATALOADERS AND TRANSFORMS
https://pytorch.org/tutorials/beginner/data_loading_tutorial.html


In [ ]:
import torch
from torchvision import transforms, utils
import os
import imageio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import time

from PIL import Image
import torchvision.transforms.functional as TF
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# image lists are opened using panda:
trainlist = pd.read_csv('train.txt', sep=' ',header=None)
print(trainlist)
print(trainlist.shape)
print(len(trainlist))
print(trainlist.iloc[3,1])

In [ ]:

class CRACK500(Dataset):
    """CRACK500 dataset."""

    def __init__(self, txt_file, root_dir, train=False, data_augmentation=False):
        """
        Args:
            txt_file (string): Path to the txt file list of image paths (eg train.txt, test.txt, val.txt).
            root_dir (string): Parent directory for the directory with all the images.
            train (boolean): True for training dataset (crop images to have size 256x256, otherwise get full image)
            data_augmentation (boolean): True for using data_augmentation during training  
        """
        self.imgslist = pd.read_csv(txt_file, sep=' ',header=None)
        self.root_dir = root_dir
        self.train = train
        self.data_augmentation = data_augmentation
    def __len__(self):
        return len(self.imgslist)

    def __getitem__(self, idx):
        # get image and mask paths
        img_name = os.path.join(self.root_dir, self.imgslist.iloc[idx, 0])
        mask_name = os.path.join(self.root_dir, self.imgslist.iloc[idx, 1])
        
        # open images using PIL and apply transform list:
        image = Image.open(img_name)
        image = transforms.ToTensor()(image)
        image = transforms.Grayscale()(image)
        image = transforms.Normalize(0.5, 0.5)(image)

        # open mask binary image using PIL:
        mask = Image.open(mask_name)
        mask = transforms.ToTensor()(mask)
        mask = mask.long().squeeze()

        if self.train: #get a subimage of size 256x256:
          if self.data_augmentation:
            
            # TODO:
            print('NOT IMPLEMENTED YET')

          else: 
            # crop center 256x256:
            h,w = mask.size()
            c1 = (h-256)//2
            c2 = (w-256)//2
            image = image[:,c1:(c1+256),c2:(c2+256)]
            mask = mask[c1:(c1+256),c2:(c2+256)]
        sample = [image, mask]          
        return sample


## Test dataloader:

In [ ]:
trainset = CRACK500(txt_file='train.txt', root_dir='', train=True, data_augmentation=False) 
image, mask = trainset[100]
print(image.size())
print(image.dtype)
print(mask.size())
print(mask.dtype)

fig, ax = plt.subplots(1, 2, figsize=(16, 8))
ax[0].imshow(image.squeeze(), cmap=plt.cm.gray)
ax[0].set_title("Image")
ax[1].imshow(mask, cmap=plt.cm.gray)
ax[1].set_title("True Mask")
fig.tight_layout()
plt.show()


testset = CRACK500(txt_file='test.txt', root_dir='', train=False, data_augmentation=False) 
image, mask = testset[200]
print(image.size())
print(image.dtype)
print(mask.size())
print(mask.dtype)

valset = CRACK500(txt_file='val.txt', root_dir='', train=False, data_augmentation=False) 
image, mask = valset[21]
print(image.size())
print(image.dtype)
print(mask.size())
print(mask.dtype)

fig, ax = plt.subplots(1, 2, figsize=(16, 8))
ax[0].imshow(image.squeeze(), cmap=plt.cm.gray)
ax[0].set_title("Image")
ax[1].imshow(mask, cmap=plt.cm.gray)
ax[1].set_title("True Mask")
fig.tight_layout()
plt.show()



In [ ]:
# check full data_loader: (sanity check, useful only to check that all images are OK)
for k, dataset in enumerate([trainset, testset, valset]):
  print(['DATASET: ', k])
  for i in range(len(dataset)):
    image, mask = trainset[i]
    # if image.shape[0] != 360:
    #   print([i,image.shape])

## U-net version with same size (as in IPOL paper)


In [ ]:
#double 3x3 convolution 
def dual_conv(in_channel, out_channel):
    conv = nn.Sequential(
        nn.Conv2d(in_channel, out_channel, kernel_size=3,padding=1,padding_mode='reflect'),
        nn.ReLU(inplace= True),
        nn.Conv2d(out_channel, out_channel, kernel_size=3,padding=1,padding_mode='reflect'),
        nn.ReLU(inplace= True),
    )
    return conv

class Unet(nn.Module):
    def __init__(self):
        super(Unet, self).__init__()

        # Left side (contracting path)
        self.dwn_conv1 = dual_conv(1, 64)
        self.dwn_conv2 = dual_conv(64, 128)
        self.dwn_conv3 = dual_conv(128, 256)
        self.dwn_conv4 = dual_conv(256, 512)
        self.dwn_conv5 = dual_conv(512, 1024)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        #Right side  (expnsion path) 
        #transpose convolution is used shown as green arrow in architecture image
        self.trans1 = nn.ConvTranspose2d(1024,512, kernel_size=2, stride= 2)
        self.up_conv1 = dual_conv(1024,512)
        self.trans2 = nn.ConvTranspose2d(512,256, kernel_size=2, stride= 2)
        self.up_conv2 = dual_conv(512,256)
        self.trans3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride= 2)
        self.up_conv3 = dual_conv(256,128)
        self.trans4 = nn.ConvTranspose2d(128,64, kernel_size=2, stride= 2)
        self.up_conv4 = dual_conv(128,64)

        #output layer
        self.out = nn.Conv2d(64, 2, kernel_size=1)

    def forward(self, image):

        #forward pass for Left side
        x1 = self.dwn_conv1(image)
        x2 = self.maxpool(x1)
        x3 = self.dwn_conv2(x2)
        x4 = self.maxpool(x3)
        x5 = self.dwn_conv3(x4)
        x6 = self.maxpool(x5)
        x7 = self.dwn_conv4(x6)
        x8 = self.maxpool(x7)
        x9 = self.dwn_conv5(x8)
        

        #forward pass for Right side
        x = self.trans1(x9)
        x = self.up_conv1(torch.cat([x,x7], 1))

        x = self.trans2(x)
        x = self.up_conv2(torch.cat([x,x5], 1))

        x = self.trans3(x)
        x = self.up_conv3(torch.cat([x,x3], 1))

        x = self.trans4(x)
        x = self.up_conv4(torch.cat([x,x1], 1))
        
        x = self.out(x)
        
        return x

unet = Unet().to(device)


## Test of untrained Unet:

In [ ]:
# Apply Unet and check sizes:
image, mask = trainset[3]
out=unet(torch.unsqueeze(image.to(device),0))
print('output size: ', out.size())
print('mask size: ',mask.size())


#Training of Unet:


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(unet.parameters())


##Define data loader for batch training:

In [ ]:
dataloader = DataLoader(trainset, batch_size=4, shuffle=True, num_workers=0)
n_train = len(trainset)

In [ ]:
iter_info = 100
since = time.time()
for epoch in range(2):
  since_epoch = time.time()
  running_loss = 0.0
  epoch_loss = 0.0
  for i, data in enumerate(dataloader, 0):
    # get the inputs; data is a list of [inputs, labels]
    images = data[0].to(device)
    masks = data[1].to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = unet(images)
    loss = criterion(outputs, masks)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    epoch_loss += loss.item()
    if i % iter_info == (iter_info-1):    # print every iter_info mini-batches
      print('[%d, %5d] loss: %.3f' %
        (epoch + 1, i + 1, running_loss / iter_info))
      running_loss = 0.0
  time_elapsed_epoch = time.time() - since_epoch    
  print('Epoch completed in {:.0f}m {:.0f}s'.format(
        time_elapsed_epoch // 60, time_elapsed_epoch % 60))
  print('Epoch loss: %.3f' % (epoch_loss / n_train))

print('Finished Training')
time_elapsed = time.time() - since
print('Training completed in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))


#Visualize some prediction (on cropped val image to be improved for performance evaluation):
Images size must be multiple of 32.






In [ ]:

i = np.random.randint(len(valset))
image, mask = valset[i]
print(image.size(), mask.size())
h,w = mask.size()
nh = (h//32)*32
nw = (w//32)*32
image = image[:,:nh,:nw]
mask = mask[:nh,:nw]
print(image.size(), mask.size())

with torch.no_grad():
  out = unet(torch.unsqueeze(image.to(device),0))
  predprob = F.softmax(out,dim=1).to('cpu')
predprob1 = predprob[0,0,:,:].squeeze()
predprob2 = predprob[0,1,:,:].squeeze()
predmask = torch.argmax(out.squeeze(), dim=0).to('cpu')

plt.figure()

fig, ax = plt.subplots(1, 3, figsize=(24, 8))
ax[0].imshow(image.squeeze(), cmap=plt.cm.gray)
ax[0].set_title("Image")
ax[1].imshow(predprob1, cmap=plt.cm.gray)
ax[1].set_title("Proba map 1")
ax[2].imshow(predprob2, cmap=plt.cm.gray)
ax[2].set_title("Proba map 2")
fig.tight_layout()
plt.show()

plt.figure()

fig, ax = plt.subplots(1, 3, figsize=(24, 8))
ax[0].imshow(image.squeeze(), cmap=plt.cm.gray)
ax[0].set_title("Image")
ax[1].imshow(predmask, cmap=plt.cm.gray)
ax[1].set_title("Predicted Mask")
ax[2].imshow(mask, cmap=plt.cm.gray)
ax[2].set_title("True Mask")
fig.tight_layout()
plt.show()



## Performance evaluation:

In [ ]:
def test_performance(unet, dataset):
  tp_count = 0
  tn_count = 0
  fp_count = 0
  fn_count = 0
  with torch.no_grad():
    for i, data in enumerate(dataset):
      image = data[0]
      mask = data[1]
      h,w = mask.size()
      nh = (h//32)*32
      nw = (w//32)*32
      image = image[:,:nh,:nw].to(device)
      mask = mask[:nh,:nw].to(device)
      out = unet(torch.unsqueeze(image,0))
      predmask = torch.argmax(out.squeeze(), dim=0)
      # count:
      tp = torch.sum(predmask*mask)
      fp = torch.sum(predmask*(1.-mask))
      tn = torch.sum((1-predmask)*(1.-mask))
      fn = torch.sum((1-predmask)*mask)
      if not (tp+fp+tn+fn == nh*nw):
        print('counting inconstancy')
      tp_count = tp_count + tp
      tn_count = tn_count + tn
      fp_count = fp_count + fp
      fn_count = fn_count + fn
      if i%100==99:
        print('test_performance: Image ', i, ' / ', len(dataset))
    print(tp_count, tn_count, fp_count, fn_count)
    precision = (tp_count/(tp_count+fp_count)).to('cpu').numpy()
    recall = (tp_count/(tp_count+fn_count)).to('cpu').numpy()
    F1score =  2. * precision * recall / (precision + recall)
    return([precision, recall, F1score])


valscores = test_performance(unet, valset)
print(valscores)



#Question 1
TODO

#Question 2
TODO

#Question 3
TODO

#Question 4
TODO